## Example course catalog crawler 

In [4]:
import urllib.request
import json
import csv
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
from selenium import webdriver
import pandas as pd
import numpy as np
import nltk 
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from string import punctuation 
stemmer = PorterStemmer()

#cornell course catalog URL
urls = ['https://classes.cornell.edu/search/roster/FA19?q=',
        'https://classes.cornell.edu/search/roster/SU19?q=',
        'https://classes.cornell.edu/search/roster/SP19?q=',
        'https://classes.cornell.edu/search/roster/FA18?q=',
        'https://classes.cornell.edu/search/roster/SU18?q=',
        'https://classes.cornell.edu/search/roster/SP18?q=']
        
#init keyword lists (cornell accepts wildcards)
normative = ['accountability', 'critical', 'democracy', 'discrimination', 'equality', 'equity',
            'ethic', 'fair', 'feminism', 'gender', 'govern', 'history', 'inequality', 'justice',
            'law', 'legal', 'liberty', 'moral', 'normative', 'philosophy', 'poltiics', 'power',
            'privacy', 'race', 'religious', 'responsible', 'right', 'security', 'social', 'society',
            'surveillance', 'transparent', 'value', 
            #following keywords are for pilot only
            'anthropology', 'economy', 'policy', 'sociology']
technical = ['ai', 'algorithm', 'analytics', 'artificial+intelligence', 'automate', 'code', 
            'computing', 'cs', 'cyber', 'data', 'digital', 'ict', 'information', 'intelligence',
            'internet', 'machine', 'ml', 'natural+language+processing', 'nlp', 'platform', 'programming',
            'robot', 'software', 'system', 'technology']  

#keyword lists (if course roster requires stemming)
#TODO 

#course search using normative keywords as queries 
for url in urls:
    for keyword in normative: 
        url_keyword = url + keyword
        response = requests.get(url_keyword)
        soup = BeautifulSoup(response.content, 'lxml')
        #extract relevant elements  (course num, course title, website, credits, description)
        keepwithnext = soup.find_all(class_="keepwithnext")
        titles = [dt.get_text() for dt in keepwithnext]  #titles and course num 
        #websites = soup.select("div dd a") 
        #print(titles, websites)
        #sites = [url.get_text() for url in websites] #course website 
        details = soup.select('div dd') 
        desc = [dd.get_text() for dd in details] #credits, description, website (need to parse/clean)
        titles.reset_index(drop=True, inplace=True)
        desc.reset_index(drop=True, inplace=True)
        df = pd.concat([titles, desc], axis=1)
   # data = data.append(pd.DataFrame({'Title': titles,'Description':desc}, index=[0]), ignore_index=False)
    #print(data.head())
    
#search for technical keywords within course list
#for course in courses:
    #for keyword in technical:
        #return courses that have union of technical + normative keywords in either title or course description

#export course list as csv
#export_csv = df.to_csv(r'crawler_response.csv',index = None,header=True) 